# Description

### Imports

In [1]:
import os
import sys
from pathlib import Path
import re
from glob import glob
import numpy as np
import pandas as pd
import torch
from torch import nn
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm
from typing import List, Dict, Tuple
from transformers import AutoTokenizer, AutoModelForTokenClassification, BertForTokenClassification, DistilBertForTokenClassification
from transformers import BertTokenizerFast, DistilBertTokenizerFast, EvalPrediction
from transformers import Trainer, TrainingArguments
from transformers import AdamW
from sklearn.metrics import confusion_matrix, classification_report
from seqeval.metrics import accuracy_score, f1_score, precision_score, recall_score

### Variables

In [2]:
#Paths 
WHD_BERT_DATA = "C:/Users/jseal/Dev/dissertation/Data/WikipediaHomographData/data/bert_data/" 
BERT_TRAIN = WHD_BERT_DATA + "train/"
BERT_VAL = WHD_BERT_DATA + "valid/"
BERT_TEST = WHD_BERT_DATA + "test/"
ALL_SPLITS = [BERT_TRAIN,BERT_VAL,BERT_TEST]



NER = "C:/Users/jseal/Dev/dissertation/transformers/examples/token-classification/"
RUN_NER = NER + "run_ner.py"

PREPROCESS = NER + "scripts/preprocess.py"

#Labels file
LABELS = WHD_BERT_DATA + "labels.txt"

# Model Variables
MAX_LENGTH = 128 #@param {type: "integer"}
OUTPUT_DIR = "C:/Users/jseal/Dev/dissertation/Models/Baselines/WHD_Bert/whd-model" #@param ["spanberta-ner", "bert-base-ml-ner"]
BATCH_SIZE = 16 #@param {type: "integer"}
NUM_EPOCHS = 3 #@param {type: "integer"}
SAVE_STEPS = 100 #@param {type: "integer"}
LOGGING_STEPS = 100 #@param {type: "integer"}
SEED = 42 #@param {type: "integer"}
EVAL_STEPS = 10

MODEL_NAME = "distilbert-base-cased"
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
print(device)

cuda:0


### Functions

In [3]:
def read_set(file_path):
    file_path = Path(file_path)

    raw_text = file_path.read_text(encoding="utf8").strip()
    raw_docs = re.split(r'\n\t?\n', raw_text)
    token_docs = []
    tag_docs = []
    for doc in raw_docs:
        tokens = []
        tags = []
        for line in doc.split('\n'):
            token, tag = line.split('\t')
            tokens.append(token)
            tags.append(tag)
        token_docs.append(tokens)
        tag_docs.append(tags)

    return token_docs, tag_docs

In [4]:
def encode_tags(tags, encodings):
    labels = [[tag2id[tag] for tag in doc] for doc in tags]
    encoded_labels = []
    for doc_labels, doc_offset in zip(labels, encodings.offset_mapping):
        # create an empty array of -100
        doc_enc_labels = np.ones(len(doc_offset),dtype=int) * -100
        arr_offset = np.array(doc_offset)

        # set labels whose first offset position is 0 and the second is not 0
        doc_enc_labels[(arr_offset[:,0] == 0) & (arr_offset[:,1] != 0)] = doc_labels
        encoded_labels.append(doc_enc_labels.tolist())

    return encoded_labels

In [5]:
class WHDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [6]:
def multi_acc(y_pred, y_test):
    y_pred_softmax = torch.log_softmax(y_pred, dim = 1)
    _, y_pred_tags = torch.max(y_pred_softmax, dim = 1)    
    
    correct_pred = (y_pred_tags == y_test).float()
    acc = correct_pred.sum() / len(correct_pred)
    
    acc = torch.round(acc) * 100
    
    return acc

In [7]:
 def align_predictions(predictions: np.ndarray, label_ids: np.ndarray) -> Tuple[List[int], List[int]]:
        preds = np.argmax(predictions, axis=2)

        batch_size, seq_len = preds.shape

        out_label_list = [[] for _ in range(batch_size)]
        preds_list = [[] for _ in range(batch_size)]

        for i in range(batch_size):
            for j in range(seq_len):
                if label_ids[i, j] != nn.CrossEntropyLoss().ignore_index:
                    out_label_list[i].append(id2tag[label_ids[i][j]])
                    preds_list[i].append(id2tag[preds[i][j]])

        return preds_list, out_label_list

In [ ]:
def compute_metrics(p: EvalPrediction) -> Dict:
    preds_list, out_label_list = align_predictions(p.predictions, p.label_ids)
    return {
        "accuracy_score": accuracy_score(out_label_list, preds_list),
        "precision": precision_score(out_label_list, preds_list),
        "recall": recall_score(out_label_list, preds_list),
        "f1": f1_score(out_label_list, preds_list),
    }

# Script

In [10]:
train_texts, train_tags = read_set(TRAIN_TXT)
val_texts, val_tags = read_set(VAL_TXT)
test_texts, test_tags = read_set(TEST_TXT)

In [11]:
unique_tags = [label.strip("\n") for label in open(LABELS).readlines()]
tag2id = {tag: id for id, tag in enumerate(unique_tags)}
id2tag = {id: tag for tag, id in tag2id.items()}

In [12]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-cased')
train_encodings = tokenizer(train_texts, is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True)
val_encodings = tokenizer(val_texts, is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True)

In [13]:
train_labels = encode_tags(train_tags, train_encodings)
val_labels = encode_tags(val_tags, val_encodings)

In [14]:
train_encodings.pop("offset_mapping") # we don't want to pass this to the model
val_encodings.pop("offset_mapping")
train_dataset = WHDataset(train_encodings, train_labels)
val_dataset = WHDataset(val_encodings, val_labels)

In [15]:
# #Model & tokenizer
# model = DistilBertForTokenClassification.from_pretrained(MODEL_NAME, num_labels=len(unique_tags))
# model.to(device)
# model.train()

# train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
# optim = AdamW(model.parameters(), lr=5e-5)

# for epoch in range(3):
#     running_loss = 0.0
#     for i, batch in enumerate(train_loader, 0):
#         optim.zero_grad()
#         input_ids = batch['input_ids'].to(device)
#         attention_mask = batch['attention_mask'].to(device)
#         labels = batch['labels'].to(device)
#         outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
#         loss = outputs[0]
        
#         # print statistics
#         running_loss += loss.item()
#         if i % 10 == 9:    # print every 10 mini-batches
#             print('[%d, %5d] loss: %.3f' %
#                   (epoch + 1, i + 1, running_loss / 10))
#             running_loss = 0.0
            
#         loss.backward()
#         optim.step()

# model.eval()

In [17]:


model = DistilBertForTokenClassification.from_pretrained(MODEL_NAME, num_labels=len(unique_tags))
model.to(device)

training_args = TrainingArguments(
    load_best_model_at_end=True,
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total # of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=10,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    evaluation_strategy="steps",
    eval_steps=EVAL_STEPS
)

trainer = Trainer(
    model=model,                         # instantiated Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,            # evaluation dataset
    compute_metrics=compute_metrics
)

Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertForTokenClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this 

In [18]:
trainer.train()

Step,Training Loss,Validation Loss,Accuracy Score,Precision,Recall,F1
10,No log,5.569069,0.007924,0.000914,0.010186,0.001678
20,No log,5.446400,0.044836,0.000839,0.008987,0.001534
30,No log,5.234553,0.284980,0.000666,0.005392,0.001186
40,No log,4.907883,0.781795,0.000000,0.000000,0.000000
50,No log,4.390033,0.929561,0.000000,0.000000,0.000000
60,No log,3.464693,0.933503,0.000000,0.000000,0.000000
70,No log,1.753605,0.933543,0.000000,0.000000,0.000000
80,No log,0.754661,0.933543,0.000000,0.000000,0.000000
90,No log,0.631022,0.933543,0.000000,0.000000,0.000000
100,No log,0.624700,0.933543,0.000000,0.000000,0.000000


C:\Users\jseal\anaconda3\envs\dissertation_huggingface\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: abstract_adj-nou seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\jseal\anaconda3\envs\dissertation_huggingface\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: abuses_nou seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\jseal\anaconda3\envs\dissertation_huggingface\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: abuses_vrb seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\jseal\anaconda3\envs\dissertation_huggingface\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: abuse_nou seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\jseal\anaconda3\envs\dissertation_huggingface\lib\site-packages\seqeval\metrics\sequence_label

C:\Users\jseal\anaconda3\envs\dissertation_huggingface\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: abstract_adj-nou seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\jseal\anaconda3\envs\dissertation_huggingface\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: abuses_nou seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\jseal\anaconda3\envs\dissertation_huggingface\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: abuses_vrb seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\jseal\anaconda3\envs\dissertation_huggingface\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: abuse_nou seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\jseal\anaconda3\envs\dissertation_huggingface\lib\site-packages\seqeval\metrics\sequence_label

C:\Users\jseal\anaconda3\envs\dissertation_huggingface\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: abstract_adj-nou seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\jseal\anaconda3\envs\dissertation_huggingface\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: abuses_nou seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\jseal\anaconda3\envs\dissertation_huggingface\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: abuses_vrb seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\jseal\anaconda3\envs\dissertation_huggingface\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: abuse_nou seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\jseal\anaconda3\envs\dissertation_huggingface\lib\site-packages\seqeval\metrics\sequence_label

C:\Users\jseal\anaconda3\envs\dissertation_huggingface\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: abstract_adj-nou seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\jseal\anaconda3\envs\dissertation_huggingface\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: abuses_nou seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\jseal\anaconda3\envs\dissertation_huggingface\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: abuses_vrb seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\jseal\anaconda3\envs\dissertation_huggingface\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: abuse_nou seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\jseal\anaconda3\envs\dissertation_huggingface\lib\site-packages\seqeval\metrics\sequence_label

...

C:\Users\jseal\anaconda3\envs\dissertation_huggingface\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: abstract_adj-nou seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\jseal\anaconda3\envs\dissertation_huggingface\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: abuses_nou seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\jseal\anaconda3\envs\dissertation_huggingface\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: abuses_vrb seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\jseal\anaconda3\envs\dissertation_huggingface\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: abuse_nou seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\jseal\anaconda3\envs\dissertation_huggingface\lib\site-packages\seqeval\metrics\sequence_label

C:\Users\jseal\anaconda3\envs\dissertation_huggingface\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: abstract_adj-nou seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\jseal\anaconda3\envs\dissertation_huggingface\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: abuses_nou seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\jseal\anaconda3\envs\dissertation_huggingface\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: abuses_vrb seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\jseal\anaconda3\envs\dissertation_huggingface\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: abuse_nou seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\jseal\anaconda3\envs\dissertation_huggingface\lib\site-packages\seqeval\metrics\sequence_label

C:\Users\jseal\anaconda3\envs\dissertation_huggingface\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: abstract_adj-nou seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\jseal\anaconda3\envs\dissertation_huggingface\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: abuses_nou seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\jseal\anaconda3\envs\dissertation_huggingface\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: abuses_vrb seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\jseal\anaconda3\envs\dissertation_huggingface\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: abuse_nou seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\jseal\anaconda3\envs\dissertation_huggingface\lib\site-packages\seqeval\metrics\sequence_label

C:\Users\jseal\anaconda3\envs\dissertation_huggingface\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: abstract_adj-nou seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\jseal\anaconda3\envs\dissertation_huggingface\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: abuses_nou seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\jseal\anaconda3\envs\dissertation_huggingface\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: abuses_vrb seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\jseal\anaconda3\envs\dissertation_huggingface\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: abuse_nou seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\jseal\anaconda3\envs\dissertation_huggingface\lib\site-packages\seqeval\metrics\sequence_label

TrainOutput(global_step=303, training_loss=1.4939581238397277)

In [19]:
trainer.evaluate()

C:\Users\jseal\anaconda3\envs\dissertation_huggingface\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: abstract_adj-nou seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\jseal\anaconda3\envs\dissertation_huggingface\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: abuses_nou seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\jseal\anaconda3\envs\dissertation_huggingface\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: abuses_vrb seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\jseal\anaconda3\envs\dissertation_huggingface\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: abuse_nou seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\jseal\anaconda3\envs\dissertation_huggingface\lib\site-packages\seqeval\metrics\sequence_label

{'eval_loss': 0.3063199520111084,
 'eval_accuracy_score': 0.9491916859122402,
 'eval_precision': 0.8016194331983806,
 'eval_recall': 0.2372678250449371,
 'eval_f1': 0.36615811373092927,
 'epoch': 3.0,
 'total_flos': 89413262538750}